In [ ]:
!curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar &> /dev/null

In [ ]:
!pip install -U antlr4-python3-runtime &> /dev/null

In [ ]:
%%file ToDoLang.g4
grammar ToDoLang;

// --------- Parser ---------

program
    : stmt* EOF
    ;

// Cada sentencia puede ser de varios tipos
stmt
    : taskDecl                    # stmtTask
    | categoryDecl                # stmtCategory
    | doneStmt                    # stmtDone
    | showStmt                    # stmtShow
    | ID ':' decltype '=' expr ';'# stmtAssign   // integración estilo compilador clásico x : int = 2 + 3 * 4;
    ;

// --------- Tareas ---------

taskDecl
    : 'task' STRING 'priority' PRIORITY ('deadline' DATE)? (statusDecl)? ';'
    ;

doneStmt
    : 'done' STRING ';'
    ;

showStmt
    : 'show' ( 'all' | 'pending' | 'done' | 'priority' PRIORITY ) ';'
    ;

categoryDecl
    : 'category' STRING '{' taskDecl* '}'
    ;

statusDecl
    : STATUS
    ;

// --------- Expresiones (inspirado en Calc) ---------

expr
    : '-' expr                        # exprNeg
    | <assoc=right> expr '^' expr     # exprPow
    | expr ('*'|'/'|'//'|'%') expr    # exprMulDiv
    | expr ('+'|'-') expr             # exprAddSub
    | '(' expr ')'                    # exprPar
    | num                             # exprLiteral
    | ID                              # exprVar
    ;

num : INT | FLOAT ;
decltype : TINT | TFLOAT ;

// --------- Lexer ---------

PRIORITY : 'low' | 'medium' | 'high' ;
STATUS   : 'done' | 'pending' ;
DATE     : [0-9]{4} '-' [0-9]{2} '-' [0-9]{2} ;
STRING   : '"' (~["\r\n"])* '"' ;

POW    : '^' ;
ADD    : '+' ;
SUB    : '-' ;
MUL    : '*' ;
DIV    : '/' ;
INTDIV : '//' ;
MOD    : '%' ;
ASSIGN : '=' ;
LPAR   : '(' ;
RPAR   : ')' ;
COLON  : ':' ;

TINT   : 'int' ;
TFLOAT : 'float' ;

ID    : [A-Za-z_][A-Za-z_0-9]* ;
INT   : [0-9]+ ;
FLOAT : [0-9]+'.'[0-9]+ ;

WS  : [ \t\n\r]+ -> skip ;
COMMENT : '//' ~[\r\n]* -> skip ;

Overwriting ToDoLang.g4


In [ ]:
!java -cp .:antlr-4.13.2-complete.jar org.antlr.v4.Tool ToDoLang.g4 -no-listener -visitor -Dlanguage=Python3

warning(180): ToDoLang.g4:60:17: chars " used multiple times in set ["\r\n"]


In [ ]:
from antlr4 import *
from ToDoLangLexer import ToDoLangLexer
from ToDoLangParser import ToDoLangParser
from ToDoLangVisitor import ToDoLangVisitor

In [ ]:
%%writefile Visitor.py
from ToDoLangParser import ToDoLangParser
from ToDoLangVisitor import ToDoLangVisitor

class Visitor(ToDoLangVisitor):
    def __init__(self):
        # Tabla de símbolos para variables aritméticas
        self.symb = {}
        # Almacenamiento de tareas
        self.tasks = {}

    # -----------------------------
    # Programa
    # -----------------------------
    def visitProgram(self, ctx:ToDoLangParser.ProgramContext):
        self.visitChildren(ctx)
        return None

    # -----------------------------
    # Tareas
    # -----------------------------
    def visitTaskDecl(self, ctx:ToDoLangParser.TaskDeclContext):
        taskName = ctx.STRING().getText().strip('"')
        priority = ctx.PRIORITY().getText()
        deadline = ctx.DATE().getText() if ctx.DATE() else None
        status = ctx.statusDecl().STATUS().getText() if ctx.statusDecl() and ctx.statusDecl().STATUS() else 'pending'


        self.tasks[taskName] = {
            "priority": priority,
            "deadline": deadline,
            "status": status
        }
        print(f"[Nueva tarea] {taskName} (priority={priority}, deadline={deadline}, status={status})")
        return None

    def visitDoneStmt(self, ctx:ToDoLangParser.DoneStmtContext):
        taskName = ctx.STRING().getText().strip('"')
        if taskName in self.tasks:
            self.tasks[taskName]["status"] = "done"
            print(f"[Update] Tarea '{taskName}' marcada como done")
        else:
            print(f"[Error] Tarea '{taskName}' no existe")
        return None

    def visitShowStmt(self, ctx:ToDoLangParser.ShowStmtContext):
        filterType = ctx.getText().replace("show", "").replace(";", "").strip()
        print(f"[Mostrar] Filtro = {filterType}")

        if not isinstance(self.tasks, dict):
            self.tasks = {}
            print("[Warning] self.tasks was not a dictionary. Initialized as empty dictionary.")

        for name, info in self.tasks.items():
            if filterType == "all":
                print(f" - {name}: {info}")
            elif filterType == "pending" and info["status"] == "pending":
                print(f" - {name}: {info}")
            elif filterType == "done" and info["status"] == "done":
                print(f" - {name}: {info}")
            elif filterType.startswith("priority") and info["priority"] in filterType:
                print(f" - {name}: {info}")
        return None

    def visitCategoryDecl(self, ctx:ToDoLangParser.CategoryDeclContext):
        categoryName = ctx.STRING().getText().strip('"')
        print(f"[Categoría] {categoryName}")
        self.visitChildren(ctx)
        return None

    # -----------------------------
    # Expresiones aritméticas
    # -----------------------------
    def visitStmtAssign(self, ctx:ToDoLangParser.StmtAssignContext):
        varType = self.visit(ctx.decltype())
        varName = ctx.ID().getText()
        exprValue, exprType  = self.visit(ctx.expr())

        if exprType == varType:
            self.symb[varName] = (exprValue, varType)
        else:
            # Casting automático
            if varType == 'int':
                self.symb[varName] = (int(exprValue), varType)
            else:
                self.symb[varName] = (float(exprValue), varType)
        print(f"[Asignación] {varName}:{varType} = {self.symb[varName][0]}")
        return None

    def visitExprNeg(self, ctx:ToDoLangParser.ExprNegContext):
        exprValue, exprType = self.visit(ctx.expr())
        return (-exprValue, exprType)

    def visitExprPar(self, ctx:ToDoLangParser.ExprParContext):
        return self.visit(ctx.expr())

    def visitExprAddSub(self, ctx:ToDoLangParser.ExprAddSubContext):
        lhsValue, lhsType = self.visit(ctx.expr(0))
        rhsValue, rhsType = self.visit(ctx.expr(1))
        if lhsType == 'float' or rhsType == 'float':
            resultType = 'float'
            lhsValue = float(lhsValue)
            rhsValue = float(rhsValue)
        else:
            resultType = 'int'
        if ctx.ADD():
            return (lhsValue + rhsValue, resultType)
        else:
            return (lhsValue - rhsValue, resultType)

    def visitExprVar(self, ctx:ToDoLangParser.ExprVarContext):
        varName = ctx.ID().getText()
        if varName not in self.symb:
            raise Exception(f'Variable {varName} no declarada')
        return self.symb[varName]

    def visitExprLiteral(self, ctx:ToDoLangParser.ExprLiteralContext):
        return self.visit(ctx.num())

    def visitNum(self, ctx:ToDoLangParser.NumContext):
        if ctx.INT():
            return (int(ctx.INT().getText()), 'int')
        else:
            return (float(ctx.FLOAT().getText()), 'float')

    def visitDecltype(self, ctx:ToDoLangParser.DecltypeContext):
        return ctx.getText()

    def visitExprMulDiv(self, ctx:ToDoLangParser.ExprMulDivContext):
        lhsValue, lhsType = self.visit(ctx.expr(0))
        rhsValue, rhsType = self.visit(ctx.expr(1))
        if ctx.INTDIV() or ctx.MOD():
            lhsValue = int(lhsValue)
            rhsValue = int(rhsValue)
            if ctx.INTDIV():
                return (lhsValue // rhsValue, 'int')
            else:
                return (lhsValue % rhsValue, 'int')

        if lhsType == 'float' or rhsType == 'float':
            resultType = 'float'
            lhsValue = float(lhsValue)
            rhsValue = float(rhsValue)
        else:
            resultType = 'int'

        if ctx.MUL():
            return (lhsValue * rhsValue, resultType)
        elif resultType == 'int':
            return (lhsValue // rhsValue, resultType)
        else:
            return (lhsValue / rhsValue, resultType)

Overwriting Visitor.py


In [ ]:
%%file ej1.xyz
task "Estudiar ANTLR" priority high deadline 2025-10-05 pending;
task "Comprar pan" priority low done;

show all;
done "Estudiar ANTLR";
show done;

x : int = 2 + 3 * 4;
y : float = x / 2.0;

Overwriting ej1.xyz


In [ ]:
from antlr4 import *
from ToDoLangLexer import ToDoLangLexer
from ToDoLangParser import ToDoLangParser

from Visitor import Visitor
import sys

input_stream = FileStream('ej1.xyz')

lexer = ToDoLangLexer(input_stream)
token_stream = CommonTokenStream(lexer)
parser = ToDoLangParser(token_stream)

tree = parser.program()

visitor = Visitor()
visitor.visit(tree)

[Nueva tarea] Estudiar ANTLR (priority=high, deadline=None, status=pending)
[Nueva tarea] Comprar pan (priority=low, deadline=None, status=pending)
[Mostrar] Filtro = all
 - Estudiar ANTLR: {'priority': 'high', 'deadline': None, 'status': 'pending'}
 - Comprar pan: {'priority': 'low', 'deadline': None, 'status': 'pending'}
[Update] Tarea 'Estudiar ANTLR' marcada como done
[Mostrar] Filtro = done
 - Estudiar ANTLR: {'priority': 'high', 'deadline': None, 'status': 'done'}
[Asignación] x:int = 14
[Asignación] y:float = 7.0


line 1:45 mismatched input '2025' expecting DATE
line 2:32 extraneous input 'done' expecting {';', 'deadline', 'status'}


In [ ]:
!java -cp .:antlr-4.13.2-complete.jar org.antlr.v4.Tool ToDoLang.g4 -no-listener -visitor -Dlanguage=Python3

warning(180): ToDoLang.g4:60:17: chars " used multiple times in set ["\r\n"]
